In [1]:
''' define the instant class and some useful scripts related to them '''

class instant:

    def __init__(instant=int, zipcode=str, coordinates=dict, forecasts=list, observation=dict):
        {'instant': instant,
        'zipcode': zipcode,
         'coordinates': coordinates,
        'forecasts': forecasts,
        'observation': observation
        }
    
    def count(instant):
        ''' Count the number of elemnets in the forecasts array '''
        
        return len(instant['forecats'])
        

def count(instant):
    ''' Count the number of elemnets in the forecasts array '''

    return len(instant['forecasts'])

def cast_count_all(col):
    ''' get a tally for the forecast counts per document 
    
    :param col: the collection you want to evaluate
    :type col: pymongo.collection.Collection
    '''
    print('getting started with cast_count_all')
    n = 0
    collection_cast_counts = {}
    # Go through each doc in the collection and count the number of items in the forecasts array.
    # Add to the tally for that count.
    for doc in col.find({}):
        n = count(doc)
        if n in collection_cast_counts:
            collection_cast_counts[n] += 1
        else:
            collection_cast_counts[n] = 1
    return collection_cast_counts

def itslegit(instant):
    ''' check the instant's weathers array for count. if it is 40, then the document is returned

    :param instant: the instant docuemnt to be legitimized
    :type instant: dictionary
    '''

    if count(instant) == 40:
        return True
    else:
        return False

def find_legit(collection):
    ''' find the 'legit' instants within the collection specified

    :param collection: database collection
    :type collection: pymongo.collection.Collection
    :return: list of documents
    '''

    return [item for item in collection.find({}) if itslegit(item)]

def load_legit(collection):
    ''' load the 'legit' instants to the remote database 

    :param collection: the collection you want to pull instants from
    :type collection: pymongo.collection.Collection
    '''
    from db_ops import Client, dbncol
    from config import uri, host, port

    legit_list = getlegit(collection)
    client = Client(host, port)
    col = dbncol(client, 'legit_inst', 'owmap')
    col.insert_many(legit_list)
    return

def test_load_legit(collection):
    ''' load the 'legit' instants to the remote database 

    :param collection: the collection you want to pull instants from
    :type collection: pymongo.collection.Collection
    '''
    from db_ops import Client, dbncol
    from config import host, port
    host = 'localhost'
    port = 27017
    legit_list = getlegit(collection)
    client = Client(host, port)
    col = dbncol(client, 'legit_inst', 'test')
    col.insert_many(legit_list)
    return

In [2]:
from pymongo import MongoClient
import db_ops
# from Extract.config import host, port
client = db_ops.Client(host=db_ops.host, port=db_ops.port)
collection = 'instant_temp'
database = 'owmap'
col = db_ops.dbncol(client, collection, database)

In [3]:
def key_list(d=dict):
    ''' Write the dict keys to a list and return that list 
    
    :param d: A python dictionary
    :return: A list fo the keys from the python dictionary
    '''
    
    keys = d.keys()
    key_list = []
    for k in keys:
        key_list.append(k)
    return key_list


key = col.find_one({})
key_list(key)

['_id', 'instant', 'zipcode', 'forecasts', 'weather']

In [23]:
def key_list(d=dict):
    ''' Write the dict keys to a list and return that list 
    
    :param d: A python dictionary
    :return: A list fo the keys from the python dictionary
    '''
    
    keys = d.keys()
    key_list = []
    for k in keys:
        key_list.append(k)
    return key_list


def all_keys(d):
    ''' Get all the dicitonary and nested "dot format" nested dictionary keys
    from a dict, add them to a list, return the list.
    
    :param d: A python dictionary
    :return: A list of every key in the dictionary
    '''
    
    top_keys = d.keys()
    i = 0  # use this as a reference to top_key values
    for key, value in d.items():
        n_key = key  # Hold on to this key as you will use it in naming any nested keys
#         for k, v in d.items():
#         print('loop')
        if isinstance(d[key], dict):
            keys.append(n_key)
            t_key = str(n_key)
                # print(f'jumped into {v}')
#             n_key = t_key+str(d[key].key())
            all_keys(d[key])
        else:
            keys.append(str(t_key) + f'.{str(n_key)}')
    return keys

d = col.find({})[1]
# print(d)
keys = []
t_key = ''  # Represent relative top dict keys
all_keys(d)
print(keys)

['_id.', 'instant.', 'zipcode.', 'forecasts.', 'weather', 'sunset_time.', 'sunrise_time.', 'clouds.', 'rain', 'snow', 'wind', 'speed.', 'deg.', 'humidity.wind', 'pressure', 'press.', 'sea_level.', 'temperature', 'temp.', 'temp_kf.', 'temp_max.', 'temp_min.', 'status.temperature', 'detailed_status.temperature', 'weather_code.temperature', 'weather_icon_name.temperature', 'visibility_distance.temperature', 'dewpoint.temperature', 'humidex.temperature', 'heat_index.temperature', 'time_to_instant.temperature']


In [8]:
key.items()
# cast_count_all(col)

dict_items([('_id', ObjectId('5e9bb25c42f97b1723883d7c')), ('instant', 1587265200), ('zipcode', '27006'), ('forecasts', [{'sunset_time': 0, 'sunrise_time': 0, 'clouds': 0, 'rain': {}, 'snow': {}, 'wind': {'speed': 1.7, 'deg': 39}, 'humidity': 55, 'pressure': {'press': 1019, 'sea_level': 1019}, 'temperature': {'temp': 280.13, 'temp_kf': -0.47, 'temp_max': 280.6, 'temp_min': 280.13}, 'status': 'Clear', 'detailed_status': 'clear sky', 'weather_code': 800, 'weather_icon_name': '01n', 'visibility_distance': None, 'dewpoint': None, 'humidex': None, 'heat_index': None, 'time_to_instant': 3178}]), ('weather', {'sunset_time': 1587254263, 'sunrise_time': 1587206633, 'clouds': 0, 'rain': {}, 'snow': {}, 'wind': {'speed': 0.45, 'deg': 40, 'gust': 1.34}, 'humidity': 48, 'pressure': {'press': 1014, 'sea_level': None}, 'temperature': {'temp': 283.22, 'temp_kf': None, 'temp_max': 284.26, 'temp_min': 281.48}, 'status': 'Clear', 'detailed_status': 'clear sky', 'weather_code': 800, 'weather_icon_name': '

In [3]:
inst_col = col.count_documents({})
print(inst_col)
cast_count_all(col)

176040
getting started with cast_count_all


{1: 2160,
 2: 3240,
 3: 2160,
 4: 2160,
 5: 2160,
 6: 2160,
 7: 2160,
 8: 2160,
 9: 2160,
 10: 2160,
 11: 4319,
 12: 3168,
 13: 7868,
 14: 4381,
 15: 7237,
 16: 2298,
 17: 2982,
 18: 3149,
 19: 4703,
 20: 2923,
 22: 3012,
 23: 4365,
 25: 3659,
 26: 3958,
 28: 3874,
 29: 2887,
 30: 3067,
 31: 4456,
 32: 5205,
 33: 15508,
 34: 19771,
 21: 3814,
 24: 4701,
 27: 2533,
 35: 13399,
 36: 10086,
 37: 5107,
 38: 928,
 39: 2}

In [4]:
cast_count_all(col)

getting started with cast_count_all


{1: 2160,
 2: 3240,
 3: 2160,
 4: 2160,
 5: 2160,
 6: 2160,
 7: 2160,
 8: 2160,
 9: 2160,
 10: 2160,
 11: 4319,
 12: 3168,
 13: 8883,
 14: 2286,
 15: 6222,
 16: 4393,
 17: 2982,
 18: 3149,
 19: 4703,
 20: 2923,
 22: 3012,
 23: 4366,
 25: 3659,
 26: 3961,
 28: 4112,
 29: 2262,
 30: 2827,
 31: 2807,
 32: 3437,
 33: 16368,
 34: 21311,
 21: 3813,
 24: 4700,
 27: 3215,
 35: 14928,
 36: 11209,
 37: 5448,
 38: 1055,
 39: 2}

In [30]:
itslegit(instant)

False

In [31]:
legit_list = find_legit(col)

In [32]:
len(legit_list)

0

In [1]:
load_legit(col)

NameError: name 'load_legit' is not defined